In [ ]:
import zipfile
import os

## --- DOWNLOAD DATASET 

# download the file 
#!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

# decompress the folder
'''local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

## --- DOWNLOAD DATASET

## -- VARIABLE DEFINITION

#  declare the location of our training and validation files

'''

base_dir = '/content/sample_data/pan_card'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

# cat and dog folders for training
train_cats_dir = os.path.join(train_dir, 'real')
train_dogs_dir = os.path.join(train_dir, 'fake')

# cat and dog folders for validation
validation_cats_dir = os.path.join(validation_dir, 'real')
validation_dogs_dir = os.path.join(validation_dir, 'fake')


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # since Conv2D is the first layer of the neural network, we should also specify the size of the input
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    # apply pooling
    tf.keras.layers.MaxPooling2D(2,2),
    # and repeat the process
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten the result to feed it to the dense layer
    tf.keras.layers.Flatten(), 
    # and define 512 neurons for processing the output coming by the previous layers
    tf.keras.layers.Dense(512, activation='relu'), 
    # a single output neuron. The result will be 0 if the image is a cat, 1 if it is a dog
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [ ]:
model.summary() 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 17, 17, 64)      

In [ ]:
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# we rescale all our images with the rescale parameter
train_datagen = ImageDataGenerator(rescale = 1.0/255)
test_datagen  = ImageDataGenerator(rescale = 1.0/255)

# we use flow_from_directory to create a generator for training
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

# we use flow_from_directory to create a generator for validation
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))

Found 16 images belonging to 3 classes.
Found 10 images belonging to 3 classes.


In [ ]:
history = model.fit(
            train_generator, # pass in the training generator
            steps_per_epoch=100,
            epochs=10,
            validation_data=validation_generator, # pass in the validation generator
            validation_steps=50,
            verbose=2
            )

Epoch 1/10


100/100 - 5s - loss: 0.9111 - accuracy: 0.0000e+00 - val_loss: -4.3524e+00 - val_accuracy: 0.5000 - 5s/epoch - 47ms/step


In [ ]:
import numpy as np

from google.colab import files
from keras.utils import load_img, img_to_array 

uploaded = files.upload()
print(uploaded)
for fn in uploaded.keys():
 
  # prediction on the uploaded image
  path = '/content/' + fn # load the image on Colab
  img =load_img(path, target_size=(150,150))# let's use load_img to scale it 
  
  # scaling process
  x = img_to_array(img)
  x /= 255 
  x = np.expand_dims(x, axis=0)
  # flatten the output
  images = np.vstack([x])
  
  # prediction!
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0] > 0.5:
    print(fn + " real")
  else:
    print(fn + " fake")

Saving sign.jpeg to sign.jpeg
{'sign.jpeg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\n\x07\x08\x16\x15\x15\x18\x16\x15\x15\x18\x19\x18\x19\x1c\x18\x1e\x1c\x1a\x1c\x1a\x1c\x1a\x19\x19\x1a!!\x1c\x1a\x1c\x18\x1c\x1a\x1c!.%\x1c!+!\x1a\x1a&8&+/1555\x1c$;@;4?.451\x01\x0c\x0c\x0c\x10\x0f\x10\x1f\x12\x12\x1e5+%+4444441=444444441144441444444444444444444444144444\xff\xc0\x00\x11\x08\x00\xc2\x01\x03\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x00\x01\x05\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x01\x02\x03\x04\x06\x00\x07\xff\xc4\x00A\x10\x00\x02\x00\x03\x05\x04\x06\x07\x05\x08\x03\x00\x03\x00\x00\x00\x01\x02\x00\x03\x11\x04\x12!1A\x05\x06Qa"2q\x81\x91\xf0\x13B\xa1\xb1\xb2\xc1\xd1\x14Rbr\x82\x07#$34C\x92\xe1\xa2\xc2\xf1Sc\xd2\xff\xc4\x00\x1a\x01\x00\x02\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x03\x00\x01\x04\x05\x06\xff\xc4\x00-\x11\x00\x02\x02\x01\x03\x03\x03\x03\x03\x05\x01\x00\x00\x00\x00\x00\x00\